In [1]:
import sys
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1740619837536_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
spark = SparkSession\
.builder\
.appName("SparkETL")\
.getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
bucket_name = 's3storageudea2025'
file_path = 'input/ds_salaries.csv'
s3_path = f"s3a://{bucket_name}/{file_path}"

try:
    df = spark.read.csv(s3_path, header=True, inferSchema=True, sep=",")

    df.show()
    df.printSchema()
    print(f"Number of rows: {df.count()}")

    # Perform further data processing here

except Exception as e:
    print(f"Error reading CSV from S3: {e}")



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|work_year|experience_level|employment_type|           job_title|salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|     2023|              SE|             FT|Principal Data Sc...| 80000|            EUR|        85847|                ES|         100|              ES|           L|
|     2023|              MI|             CT|         ML Engineer| 30000|            USD|        30000|                US|         100|              US|           S|
|     2023|              MI|             CT|         ML Engineer| 25500|            USD|        25500|                US|         100|              US|           S|
|     2023

In [8]:
# Create a list of columns with their corresponding null counts
null_counts = df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])

# Display the null counts
null_counts.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+----------------+---------------+---------+------+---------------+-------------+------------------+------------+----------------+------------+
|work_year|experience_level|employment_type|job_title|salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---------+----------------+---------------+---------+------+---------------+-------------+------------------+------------+----------------+------------+
|        0|               0|              0|        0|     0|              0|            0|                 0|           0|               0|           0|
+---------+----------------+---------------+---------+------+---------------+-------------+------------------+------------+----------------+------------+

In [ ]:
from pyspark.sql.functions import max, min, avg
from pyspark.sql.types import NumericType 

# Filter numeric columns
numeric_cols = [col for col, dtype in df.dtypes if isinstance(df.schema[col].dataType, NumericType)]

# Calculate max, min, and average for each numeric column
stats = df.agg(
    *(max(col).alias(f"{col}_max") for col in numeric_cols),
    *(min(col).alias(f"{col}_min") for col in numeric_cols),
    *(avg(col).alias(f"{col}_avg") for col in numeric_cols)
)

# Show the results in a friendly way
stats.show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+----------+-----------------+----------------+-------------+----------+-----------------+----------------+------------------+------------------+------------------+------------------+
|work_year_max|salary_max|salary_in_usd_max|remote_ratio_max|work_year_min|salary_min|salary_in_usd_min|remote_ratio_min|work_year_avg     |salary_avg        |salary_in_usd_avg |remote_ratio_avg  |
+-------------+----------+-----------------+----------------+-------------+----------+-----------------+----------------+------------------+------------------+------------------+------------------+
|2023         |30400000  |450000           |100             |2020         |6000      |5132             |0               |2022.3736351531293|190695.57177097205|137570.38988015978|46.271637816245004|
+-------------+----------+-----------------+----------------+-------------+----------+-----------------+----------------+------------------+------------------+------------------+------------------+

Es cierto que los empleos que se pagan en euros son mejor remunerados que los pagados
en dólares

In [22]:
# Filter for USD and EUR salaries
usd_eur_salaries = df.filter(df.salary_currency.isin("USD", "EUR"))

# Calculate the average salary in USD and EUR by year
average_salaries_by_year = usd_eur_salaries.groupBy("work_year", "salary_currency").agg(avg("salary_in_usd").alias("average_salary"))

# Format the average salary to 2 decimal places
formatted_avg_salaries_by_year = average_salaries_by_year.withColumn("average_salary", format_number("average_salary", 2))

# Sort by year and then by currency
sorted_avg_salaries = formatted_avg_salaries_by_year.orderBy("work_year", "salary_currency")

# Show the results in a friendly way
sorted_avg_salaries.show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+---------------+--------------+
|work_year|salary_currency|average_salary|
+---------+---------------+--------------+
|2020     |EUR            |59,907.62     |
|2020     |USD            |129,450.26    |
|2021     |EUR            |69,316.43     |
|2021     |USD            |123,831.11    |
|2022     |EUR            |59,825.43     |
|2022     |USD            |144,821.95    |
|2023     |EUR            |62,846.38     |
|2023     |USD            |155,634.79    |
+---------+---------------+--------------+

El promedio de salarios en USD es mucho mayor al de Euros a través de los años por lo que según los datos los salarios en dólares son mejor remunerados

2. Cual es el promedio de salario por tamaño de empresa

In [23]:
# Calcular el promedio de salario por tamaño de empresa
average_salary_by_company_size = df.groupBy("company_size").agg(avg("salary_in_usd").alias("average_salary"))

# Formatear el promedio de salario a 2 decimales
formatted_avg_salaries_by_company_size = average_salary_by_company_size.withColumn("average_salary", format_number("average_salary", 2))

# Mostrar los resultados de forma amigable
formatted_avg_salaries_by_company_size.show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------+
|company_size|average_salary|
+------------+--------------+
|M           |143,130.55    |
|L           |118,300.98    |
|S           |78,226.68     |
+------------+--------------+

3. Son los empleos remotos los mejor remunerados

In [25]:
# Calcular el promedio de salario según la proporción de trabajo remoto
average_salary_by_remote_ratio = df.groupBy("remote_ratio").agg(avg("salary_in_usd").alias("average_salary"))

# Formatear el promedio de salario a 2 decimales
formatted_avg_salaries_by_remote_ratio = average_salary_by_remote_ratio.withColumn("average_salary", format_number("average_salary", 2))

# Mostrar los resultados de forma amigable
formatted_avg_salaries_by_remote_ratio.show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------+
|remote_ratio|average_salary|
+------------+--------------+
|100         |136,481.45    |
|0           |144,316.20    |
|50          |78,400.69     |
+------------+--------------+

En promedio los trabajos presenciales son un poco mejor pago que los full remoto, sin embargo los híbridos son los que son menos pagos

4. Top 10 de las profesiones con salarios mas altos y mas bajos

In [46]:

print("Top 10 profesiones mejor pagadas:")
df.createOrReplaceTempView("EMP")
spark.sql("""
    select 
        job_title,
        round(AVG(salary_in_usd),2) as avg_salary
    from EMP
    GROUP BY job_title
    ORDER BY avg_salary DESC
    LIMIT 10
    """).show(truncate=False)



print("Top 10 profesiones peor pagadas:")
spark.sql("""
    select 
        job_title,
        round(AVG(salary_in_usd),2) as avg_salary
    from EMP
    GROUP BY job_title
    ORDER BY avg_salary ASC
    LIMIT 10
    """).show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Top 10 profesiones mejor pagadas:
+----------------------------------+----------+
|job_title                         |avg_salary|
+----------------------------------+----------+
|Data Science Tech Lead            |375000.0  |
|Cloud Data Architect              |250000.0  |
|Data Lead                         |212500.0  |
|Data Analytics Lead               |211254.5  |
|Principal Data Scientist          |198171.13 |
|Director of Data Science          |195140.73 |
|Principal Data Engineer           |192500.0  |
|Machine Learning Software Engineer|192420.0  |
|Data Science Manager              |191278.78 |
|Applied Scientist                 |190264.48 |
+----------------------------------+----------+

Top 10 profesiones peor pagadas:
+----------------------------------+----------+
|job_title                         |avg_salary|
+----------------------------------+----------+
|Power BI Developer                |5409.0    |
|Product Data Scientist            |8000.0    |
|Staff Data Analyst 

5. Países en los que se presenta el mayor y el menor salario

In [47]:
print("Paises con mayor salario:")
spark.sql("""
    select 
        company_location,
        round(AVG(salary_in_usd),2) as avg_salary
    from EMP
    GROUP BY company_location
    ORDER BY avg_salary desc
    LIMIT 5
    """).show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Paises con mayor salario:
+----------------+----------+
|company_location|avg_salary|
+----------------+----------+
|IL              |271446.5  |
|PR              |167500.0  |
|US              |151822.01 |
|RU              |140333.33 |
|CA              |131917.69 |
+----------------+----------+

In [48]:
print("Paises con menor salario:")
spark.sql("""
    select 
        company_location,
        round(AVG(salary_in_usd),2) as avg_salary
    from EMP
    GROUP BY company_location
    ORDER BY avg_salary asc
    LIMIT 5
    """).show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Paises con menor salario:
+----------------+----------+
|company_location|avg_salary|
+----------------+----------+
|MK              |6304.0    |
|BO              |7500.0    |
|MA              |10000.0   |
|AL              |10000.0   |
|VN              |12000.0   |
+----------------+----------+